# AI Algorithms
> By Sayed Afnan Khazi(01JST21CB036), Venkat Bhaskar(01JST21CB049), G Rutvik(01JST21CB012), Sai Sujith(01JST21CB033)
- This notebook showcases the implementation of Artificial Neural Networks, K-Nearest Neighbors, Decision Trees, Random Forest, Support Vector Machines, and Reinforcement Learning (Q-learning) on a financial dataset containing Alibaba's stock prices for the past many years.

### Importing Libraries

In [1]:
# Install all the requirements
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing all the requirements. 
import numpy as np
import pandas as pd
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

2024-05-31 00:00:50.210600: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
final_results_table = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

### Algorithm 1: Artificial Neural Networks

In [4]:
# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize the ANN
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the ANN
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the ANN
    model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)
    
    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for test size ratio {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_movement = model.predict(last_data_point_scaled)
predicted_movement = (predicted_movement > 0.5).astype(int)

# Interpret the result
movement_label = "Eligible" if predicted_movement[0][0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next person is: {movement_label}')


# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['ANN', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

122/122 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix for ratio 0.4:
 [[3010  374]
 [ 432   68]]
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix for ratio 0.3:
 [[2304  223]
 [ 331   55]]
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Confusion Matrix for ratio 0.2:
 [[1485  206]
 [ 219   32]]
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Confusion Matrix for ratio 0.1:
 [[797  41]
 [118  15]]

Metrics for test size ratio 0.4:
Precision: 0.15
Accuracy: 0.79
Recall: 0.14
F1 Score: 0.14

Metrics for test size ratio 0.3:
Precision: 0.20
Accuracy: 0.81
Recall: 0.14
F1 Score: 0.17

Metrics for test size ratio 0.2:
Precision: 0.13
Accuracy: 0.78
Recall: 0.13
F1 Score: 0.13

Metrics for test size ratio 0.1:
Precision: 0.27
Accuracy: 0.84
Recall: 0.11
F1 Score: 0.16

Classification Report for the last ratio:
               precision    recall  f1-score   support

           0       0.87      0.95      0.91       838
           1       0.27      0.11      0.16       133

    accuracy              

### Algorithm 2: K-Nearest Neighbors

In [5]:
# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize and train the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=2)
    knn.fit(X_train, y_train)
    
    # Predictions
    y_pred = knn.predict(X_test)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)
    
    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for test size ratio {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_eligibility = knn.predict(last_data_point_scaled)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next person is: {eligibility_label}')


# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['KNN', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

Confusion Matrix for ratio 0.4:
 [[3315   69]
 [ 483   17]]
Confusion Matrix for ratio 0.3:
 [[2477   50]
 [ 376   10]]
Confusion Matrix for ratio 0.2:
 [[1655   36]
 [ 242    9]]
Confusion Matrix for ratio 0.1:
 [[826  12]
 [128   5]]

Metrics for test size ratio 0.4:
Precision: 0.20
Accuracy: 0.86
Recall: 0.03
F1 Score: 0.06

Metrics for test size ratio 0.3:
Precision: 0.17
Accuracy: 0.85
Recall: 0.03
F1 Score: 0.04

Metrics for test size ratio 0.2:
Precision: 0.20
Accuracy: 0.86
Recall: 0.04
F1 Score: 0.06

Metrics for test size ratio 0.1:
Precision: 0.29
Accuracy: 0.86
Recall: 0.04
F1 Score: 0.07

Classification Report for the last ratio:
               precision    recall  f1-score   support

           0       0.87      0.99      0.92       838
           1       0.29      0.04      0.07       133

    accuracy                           0.86       971
   macro avg       0.58      0.51      0.49       971
weighted avg       0.79      0.86      0.80       971

The predicted eligibi

### Algorithm 3: Decision Trees

In [6]:
# Load data from the new CSV file
df = pd.read_csv('dataset.csv')

# List of categorical features to encode
categorical_features = ['Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Feature selection - excluding 'ID' and 'Target'
features = [col for col in df.columns if col not in ['ID', 'Target']]
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)

    # Model training
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)

    # Predictions
    y_pred = classifier.predict(X_test)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred)
    f1_q = f1_score(y_test, y_pred)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Predict the eligibility for the next individual
predicted_eligibility = classifier.predict(last_data_point)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next individual is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['Decision Tree', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

Confusion Matrix for ratio 0.4:
 [[2873  511]
 [ 422   78]]
Confusion Matrix for ratio 0.3:
 [[2148  379]
 [ 320   66]]
Confusion Matrix for ratio 0.2:
 [[1427  264]
 [ 202   49]]
Confusion Matrix for ratio 0.1:
 [[729 109]
 [108  25]]

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.87      0.87       838
           1       0.19      0.19      0.19       133

    accuracy                           0.78       971
   macro avg       0.53      0.53      0.53       971
weighted avg       0.78      0.78      0.78       971

The predicted eligibility for the next individual is: Not Eligible
Results:
Ratio Precision Accuracy Recall F1
0.4	0.1324	0.7598	0.1560	0.1433
0.3	0.1483	0.7600	0.1710	0.1588
0.2	0.1565	0.7600	0.1952	0.1738
0.1	0.1866	0.7765	0.1880	0.1873
Average F1 0.1658
Average Accuracy 0.7641
Average Precision 0.156
Average Recall 0.1775


### Algorithm 4: Random Forest

In [7]:
# Load data from the new CSV file
df = pd.read_csv('dataset.csv')

# List of categorical features to encode
categorical_features = ['Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Feature selection - excluding 'ID' and 'Target'
features = [col for col in df.columns if col not in ['ID', 'Target']]
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)

    # Model training
    classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)

    # Predictions
    y_pred = classifier.predict(X_test)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred)
    f1_q = f1_score(y_test, y_pred)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Predict the eligibility for the next individual
predicted_eligibility = classifier.predict(last_data_point)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next individual is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['Random Forest', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

Confusion Matrix for ratio 0.4:
 [[3382    2]
 [ 499    1]]
Confusion Matrix for ratio 0.3:
 [[2523    4]
 [ 384    2]]
Confusion Matrix for ratio 0.2:
 [[1687    4]
 [ 250    1]]
Confusion Matrix for ratio 0.1:
 [[838   0]
 [133   0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.86      1.00      0.93       838
           1       0.00      0.00      0.00       133

    accuracy                           0.86       971
   macro avg       0.43      0.50      0.46       971
weighted avg       0.74      0.86      0.80       971

The predicted eligibility for the next individual is: Not Eligible
Results:
Ratio Precision Accuracy Recall F1
0.4	0.3333	0.8710	0.0020	0.0040
0.3	0.3333	0.8668	0.0052	0.0102
0.2	0.2000	0.8692	0.0040	0.0078
0.1	0.0000	0.8630	0.0000	0.0000
Average F1 0.0055
Average Accuracy 0.8675
Average Precision 0.2167
Average Recall 0.0028


/Users/afnan/Programming/compare-algos-ai-class/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/afnan/Programming/compare-algos-ai-class/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/afnan/Programming/compare-algos-ai-class/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

### Algorithm 5: Support Vector Machines

In [8]:
# Load data from a CSV file
df = pd.read_csv('dataset.csv')

# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=0)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Initialize and train the SVM classifier
    svm_classifier = SVC(kernel='sigmoid', random_state=0)
    svm_classifier.fit(X_train, y_train)
    
    # Predictions
    y_pred = svm_classifier.predict(X_test)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for ratio {ratio}:\n", cm)
    
    # Calculate metrics
    precision_q = precision_score(y_test, y_pred, zero_division=0)
    accuracy_q = accuracy_score(y_test, y_pred)
    recall_q = recall_score(y_test, y_pred, zero_division=0)
    f1_q = f1_score(y_test, y_pred, zero_division=0)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Display results
for ratio, metrics in results.items():
    print(f"\nMetrics for test size ratio {ratio}:")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Accuracy: {metrics['accuracy']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Detailed classification report for the last test size ratio
print("\nClassification Report for the last ratio:\n", classification_report(y_test, y_pred, zero_division=0))

# Get the most recent data point and format it as a DataFrame
last_data_point = df[features].iloc[-1].to_frame().T

# Standardize the last data point
last_data_point_scaled = scaler.transform(last_data_point)

# Predict the eligibility for the next data point
predicted_eligibility = svm_classifier.predict(last_data_point_scaled)

# Interpret the result
eligibility_label = "Eligible" if predicted_eligibility[0] == 1 else "Not Eligible"
print(f'The predicted eligibility for the next data point is: {eligibility_label}')

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['SVM',
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

Confusion Matrix for ratio 0.4:
 [[3269  115]
 [ 479   21]]
Confusion Matrix for ratio 0.3:
 [[2454   73]
 [ 370   16]]
Confusion Matrix for ratio 0.2:
 [[1605   86]
 [ 239   12]]
Confusion Matrix for ratio 0.1:
 [[791  47]
 [123  10]]

Metrics for test size ratio 0.4:
Precision: 0.15
Accuracy: 0.85
Recall: 0.04
F1 Score: 0.07

Metrics for test size ratio 0.3:
Precision: 0.18
Accuracy: 0.85
Recall: 0.04
F1 Score: 0.07

Metrics for test size ratio 0.2:
Precision: 0.12
Accuracy: 0.83
Recall: 0.05
F1 Score: 0.07

Metrics for test size ratio 0.1:
Precision: 0.18
Accuracy: 0.82
Recall: 0.08
F1 Score: 0.11

Classification Report for the last ratio:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90       838
           1       0.18      0.08      0.11       133

    accuracy                           0.82       971
   macro avg       0.52      0.51      0.50       971
weighted avg       0.77      0.82      0.79       971

The predicted eligibi

### Algorithm 6: Reinforcement Learning (Q-learning)

In [9]:
# Seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load data from a CSV file
file_path = 'dataset.csv'
df = pd.read_csv(file_path)


# Preprocess data
# Handling missing values (if any)
df = df.dropna()

# Convert categorical columns to numerical values using one-hot encoding
categorical_columns = ['Gender', 'Own_car', 'Own_property', 'Work_phone', 'Phone', 'Email', 'Income_type', 'Education_type', 'Family_status', 'Housing_type', 'Occupation_type']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Feature selection
# Assuming 'Target' is the target variable
features = df.columns.difference(['ID', 'Target'])
X = df[features]
y = df['Target']

# Q-learning algorithm
def q_learning_train(X, y, episodes=1000, learning_rate=0.1, discount_factor=0.95, epsilon=0.1):
    n_actions = 2  # Eligible or Not Eligible (0 or 1)
    n_states = X.shape[0]
    
    # Initialize Q-table with zeros
    Q = np.zeros((n_states, n_actions))
    
    for _ in range(episodes):
        state = random.randint(0, n_states - 1)
        while True:
            if random.uniform(0, 1) < epsilon:  # epsilon is our exploration rate
                action = random.randint(0, n_actions - 1)  # Explore
            else:
                action = np.argmax(Q[state, :])  # Exploit
            
            reward = 1 if y.iloc[state] == action else -1
            
            next_state = (state + 1) % n_states
            Q[state, action] = Q[state, action] + learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            
            state = next_state
            if state == 0:
                break
    
    return Q

def q_learning_predict(Q, X):
    '''Predicts the actions for each state in X (test data) using the Q-table.'''
    y_pred = []
    for state in range(X.shape[0]):
        action = np.argmax(Q[state, :])
        y_pred.append(action)
    return np.array(y_pred)

def print_confusion_matrix(cm, title):
    print(f"{title}")
    print(cm,end='\n\n')

# training-testing ratios
ratios = [0.4, 0.3, 0.2, 0.1]

results = {}

for ratio in ratios:
    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)

    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train Q-learning model
    Q = q_learning_train(pd.DataFrame(X_train), y_train)
    y_pred_q = q_learning_predict(Q, pd.DataFrame(X_test))

    # Print confusion matrix
    cm = confusion_matrix(y_test, y_pred_q)
    print_confusion_matrix(cm, f"Confusion Matrix for ratio {ratio}")

    # Calculate metrics
    precision_q = precision_score(y_test, y_pred_q, zero_division=1)
    accuracy_q = accuracy_score(y_test, y_pred_q)
    recall_q = recall_score(y_test, y_pred_q)
    f1_q = f1_score(y_test, y_pred_q)
    
    results[ratio] = {'precision': precision_q, 'accuracy': accuracy_q, 'recall': recall_q, 'f1': f1_q}

# Print results
for ratio in results:
    print(f"Results for test size ratio {ratio}:")
    for metric, value in results[ratio].items():
        print(f"{metric.capitalize()}: {value:.2f}")

# Print results
print("Results:")
print("Ratio Precision Accuracy Recall F1")
for ratio, metrics in results.items():
    print(f"{ratio:}\t{metrics['precision']:.4f}\t{metrics['accuracy']:.4f}\t{metrics['recall']:.4f}\t{metrics['f1']:.4f}")
print("Average F1",round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Accuracy",round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Precision",round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4))
print("Average Recall",round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4))

final_results_table.loc[len(final_results_table)] = ['RL/Q-Learning', 
                                                     round(sum([ results[ratio]['accuracy'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['precision'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['recall'] for ratio in results.keys() ])/len(results.keys()),4),
                                                     round(sum([ results[ratio]['f1'] for ratio in results.keys() ])/len(results.keys()),4)
                                                    ]

Confusion Matrix for ratio 0.4
[[2939  402]
 [ 461   82]]

Confusion Matrix for ratio 0.3
[[2193  306]
 [ 363   51]]

Confusion Matrix for ratio 0.2
[[1439  220]
 [ 244   39]]

Confusion Matrix for ratio 0.1
[[708 127]
 [116  20]]

Results for test size ratio 0.4:
Precision: 0.17
Accuracy: 0.78
Recall: 0.15
F1: 0.16
Results for test size ratio 0.3:
Precision: 0.14
Accuracy: 0.77
Recall: 0.12
F1: 0.13
Results for test size ratio 0.2:
Precision: 0.15
Accuracy: 0.76
Recall: 0.14
F1: 0.14
Results for test size ratio 0.1:
Precision: 0.14
Accuracy: 0.75
Recall: 0.15
F1: 0.14
Results:
Ratio Precision Accuracy Recall F1
0.4	0.1694	0.7778	0.1510	0.1597
0.3	0.1429	0.7703	0.1232	0.1323
0.2	0.1506	0.7611	0.1378	0.1439
0.1	0.1361	0.7497	0.1471	0.1413
Average F1 0.1443
Average Accuracy 0.7647
Average Precision 0.1497
Average Recall 0.1398


### Our final comparison table tabulating our results

In [10]:
final_results_table

,Model,Accuracy,Precision,Recall,F1 Score
0,ANN,0.8049,0.1885,0.1297,0.1499
1,KNN,0.8561,0.2146,0.0333,0.0576
2,Decision Tree,0.7641,0.1560,0.1775,0.1658
3,Random Forest,0.8675,0.2167,0.0028,0.0055
4,SVM,0.8381,0.1580,0.0516,0.0769
5,RL/Q-Learning,0.7647,0.1497,0.1398,0.1443
